# Multi Year score
Given a target year, it computes a score for each image year of adquisition

This score is a constant value for the whole scene (not per-pixel)

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import ui, tools

In [3]:
from geebap import scores, satcol
from geebap.season import Season

In [4]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

In [5]:
col = satcol.Collection.Landsat8USGS()

In [6]:
colEE = col.colEE.filterBounds(aoi).map(col.ndvi)

In [7]:
start = '2015-11-15'
end = '2019-02-15'

In [8]:
colEE = colEE.filterDate(start, end)

In [9]:
year = 2018

In [10]:
years = list(range(2015, 2020))

## Score Linear

In [18]:
season = Season.Growing_South()

In [19]:
score = scores.MultiYear(season=season, main_year=year)

In [20]:
# `map` method cannot be tested because the year_property is added in the BAP process
# newcol = score.map(colEE, years=years)
newcol = score.apply(colEE, target_year=year)

In [21]:
chart = ui.chart.Image.series(newcol, p, scale=30, bands=['year_score'], label_bands=['linear'])

## Score Gauss

In [32]:
newcol_n = score.apply(colEE, target_year=year, function='gauss')

In [28]:
# newcol_n = score.map(colEE, year=year)

In [33]:
chart_n = ui.chart.Image.series(newcol_n, p, scale=30, bands=['year_score'], label_bands=['gauss stretch 1'])

## PLOT

In [34]:
both = chart.cat(chart_n)
both.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [35]:
both.dataframe

,gauss stretch 1,linear
2015-11-21 14:30:02.220,0.000000,0.000000
2015-11-21 14:30:26.190,0.000000,0.000000
2015-11-30 14:24:15.630,0.000000,0.000000
2015-12-07 14:30:01.020,0.000000,0.000000
2015-12-07 14:30:24.980,0.000000,0.000000
2015-12-16 14:24:14.720,0.000000,0.000000
2015-12-23 14:30:01.880,0.000000,0.000000
2015-12-23 14:30:25.850,0.000000,0.000000
2016-01-01 14:24:12.810,0.125622,0.333333
2016-01-08 14:29:57.180,0.125622,0.333333
